## 1. Realizar Análisis de Intervalos de Confianza

El análisis de intervalos de confianza nos permite estimar el rango dentro del cual se espera que se encuentre una determinada métrica poblacional, con un nivel de confianza específico. Este paso es crucial para inferir propiedades de la población a partir de nuestra muestra de datos.

### Intervalo de Confianza para la Media con Varianza Conocida

Cuando la varianza de la población (σ^2) es conocida, el intervalo de confianza para la media puede ser calculado utilizando la distribución Normal. La fórmula para un intervalo de confianza se da por:

$$\bar{x} \pm Z_{\frac{\alpha}{2}} \left(\frac{\sigma}{\sqrt{n}}\right)$$

Donde:

- $\bar{x}$: es la media de la muestra,
- $Z_{\frac{\alpha}{2}}$: es el estimador Z de la distribución normal estándar para el nivel de confianza deseado,
- $\sigma$: es la desviación estándar de la población (raíz cuadrada de la varianza),
- $n$: es el tamaño de la muestra.


In [1]:
import numpy as np
import scipy.stats as stats

def ci_mean_known_variance(data, sigma, confidence=0.95):
    n = len(data)
    mean = np.mean(data)
    z = stats.norm.ppf((1 + confidence) / 2)
    margin_error = z * (sigma / np.sqrt(n))
    return mean - margin_error, mean + margin_error

### Intervalo de Confianza para la Media con Varianza Desconocida

Cuando la varianza es desconocida, se utiliza la distribución t en su lugar.

**Fórmula:**

$$\bar{x} \pm t_{\frac{\alpha}{2}, n-1} \times \sqrt{\frac{s^2}{n}}$$

Donde:

- $s^2$ es la varianza de la muestra,
- $t_{\frac{\alpha}{2}, n-1}$ es el puntaje t de la distribución t para el nivel de confianza deseado y $n-1$ grados de libertad.

In [ ]:
from scipy.stats import t

def ci_mean_unknown_variance(sample_mean, sample_std, n, confidence_level=0.95):
    alpha = 1 - confidence_level
    df = n - 1
    t_score = t.ppf(1 - alpha/2, df)
    margin_error = t_score * (sample_std / (n**0.5))
    return (sample_mean - margin_error, sample_mean + margin_error)

### Intervalo de Confianza para la Varianza

El intervalo de confianza para la varianza se basa en la distribución chi-cuadrado.

**Fórmula:**

$$\left(\frac{(n-1)s^2}{\chi^2_{1-\frac{\alpha}{2}, n-1}}, \frac{(n-1)s^2}{\chi^2_{\frac{\alpha}{2}, n-1}}\right)$$

Donde:

- $n$ es el tamaño de la muestra,
- $s^2$ es la varianza de la muestra,
- $\chi^2_{\frac{\alpha}{2}, n-1}$ y $\chi^2_{1-\frac{\alpha}{2}, n-1}$ son los valores críticos de la distribución chi-cuadrado para el nivel de confianza deseado y $n-1$ grados de libertad.

In [ ]:
from scipy.stats import chi2

def ci_variance(data, confidence=0.95):
    n = len(data)
    var = np.var(data, ddof=1)
    chi2_lower = stats.chi2.ppf((1 - confidence) / 2, n - 1)
    chi2_upper = stats.chi2.ppf((1 + confidence) / 2, n - 1)
    lower = (n - 1) * var / chi2_upper
    upper = (n - 1) * var / chi2_lower
    return lower, upper

### Intervalo de Confianza para la Proporción

Para una proporción de muestra, la fórmula utiliza la aproximación normal.

**Fórmula:**

$$\hat{p} \pm Z_{\frac{\alpha}{2}} \times \sqrt{\frac{\hat{p}(1 - \hat{p})}{n}}$$

Donde:

- $\hat{p}$ es la proporción de la muestra.


In [ ]:
def ci_proportion(n_success, n_trials, confidence=0.95):
    p_hat = n_success / n_trials
    z = stats.norm.ppf((1 + confidence) / 2)
    margin_error = z * np.sqrt(p_hat * (1 - p_hat) / n_trials)
    return p_hat - margin_error, p_hat + margin_error

## 2. Realizar pruebas de hipotesis para una poblacion

### Media con Varianza Conocida

- **Hipótesis Nula (H0):** $ \mu = \mu_0 $ (La media de la población es igual a un valor específico $ \mu_0 $)
- **Hipótesis Alternativa (H1):** $ \mu \neq \mu_0 $ (La media de la población es diferente de $ \mu_0 $)

Para este caso, se utiliza la distribución normal si el tamaño de la muestra es grande ($ n>30 $). Si la varianza poblacional es conocida, se puede aplicar la prueba Z.

In [ ]:
import numpy as np
import scipy.stats as stats

def mean_known_variance_test(data, mu_0, sigma, n):
    
    # Z-test
    z_stat = (np.mean(data) - mu_0) / (sigma / np.sqrt(n))
    p_value = 2 * stats.norm.cdf(-np.abs(z_stat))

    print(f'Z-statistic: {z_stat}, P-value: {p_value}')

### Media con Varianza Desconocida

- **Hipótesis Nula (H0):** $ \mu = \mu_0 $
- **Hipótesis Alternativa (H1):** $ \mu \neq \mu_0 $

Cuando la varianza es desconocida y el tamaño de la muestra es pequeño, se utiliza la distribución t de Student.


In [ ]:
from scipy.stats import t


def mean_unknown_variance_test(data, mu_0, mu, n):
    
    s = np.std(data, ddof=1)  # standar deviation
    t_stat = (mu- mu_0) / (s / np.sqrt(n))
    
    # Compute p-value
    p_value = 2 * (1 - t.cdf(np.abs(t_stat), df=n-1))

    # Print result
    print(f'T-statistic: {t_stat}, p-value: {p_value}')

###  Varianza

- **Hipótesis Nula (H0):** $ \sigma^2 = \sigma_0^2 $
- **Hipótesis Alternativa (H1):** $ \sigma^2 \neq \sigma_0^2 $

La prueba para la varianza se basa en la distribución chi-cuadrada.

In [ ]:
from scipy.stats import chi2

def variance_test(data, sigma_0, n):
    sigma = np.var(data, ddof=1)
    chi2_stat = (n - 1) * sigma / sigma_0

    # Compute p-value
    p_value = 1 - chi2.cdf(chi2_stat, df=n-1)

    # Imprimir el resultado
    print(f'Chi2-statistic: {chi2_stat}, p-value: {p_value}')

### Proporción

- **Hipótesis Nula (H0):** $ p = p_0 $
- **Hipótesis Alternativa (H1):** $ p \neq p_0 $

Para proporciones, se utiliza la aproximación normal de la distribución binomial.

In [ ]:
from scipy.stats import norm

def proportion_test(p_0, p, n):
    
    z = (p_0 - p) / np.sqrt(p * (1 - p) / n)

    # Calcular p-valor
    p_valor = 2 * (1 - norm.cdf(np.abs(z)))

    # Imprimir el resultado
    print(f'Z-statistic: {z}, p-value: {p_valor}')